In [7]:
import json
import os

outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)
with open('bus_routes.json', 'r') as f:
  data = json.load(f)

In [8]:
service = "855"
direction = "2"
route = data[service][direction]

In [9]:
od = []
for seq, id in enumerate(route[:-1]):
    next_stop = seq + 1
    for stop in route[next_stop:]:
        if id['code'] == stop['code']:
            break #we've come back to the same bus stop. assume all pax from here on out boarded on 2nd visit.
        od.append((int(id['code']), int(stop['code'])))

To use:
Copy origin_destination_summary from the data folders into the working directory, rename to "od.csv" or change filename below

This code also works with byhour files but requires manual fixing:
- Remove line 1 (it just says `TOTAL_TRIPS` which is useless to us)
- Replace this part of line 2 `TIME_PER_HOUR,,,0,1` with `ORIGIN_PT_CODE,DESTINATION_PT_CODE,DAY_TYPE,0,1`
- Delete line 3

In [10]:
import pandas as pd
df_source = pd.read_csv("od.csv")
#df_source = pd.read_csv("od.csv")
pd.set_option('display.max_columns', None)

In [12]:
df = pd.DataFrame()
for src, dest in od:
    df = pd.concat([df, df_source[(df_source['ORIGIN_PT_CODE'] == src) & (df_source['DESTINATION_PT_CODE'] == dest)]])

#retrieve the filtered report for the given service and direction
df.to_csv(f"output/{service}_{direction}.csv")

In [13]:
df_wd = df[df["DAY_TYPE"] == "WEEKDAY"]
pd.pivot_table(df, index="ORIGIN_PT_CODE", columns="DESTINATION_PT_CODE")

TOTAL_TRIPS                                               \
DESTINATION_PT_CODE       10081 10091 10101  10111  10131  10361 11021 11031   
ORIGIN_PT_CODE                                                                 
10081                       NaN  77.5  42.5   76.5   43.5    NaN  19.0   7.0   
10091                       NaN   NaN  41.5  184.0  104.5    NaN  71.5  16.5   
10101                       NaN   NaN   NaN    9.5   30.0    NaN  17.5   1.0   
10111                       NaN   NaN   NaN    NaN    6.0    NaN  29.5   4.5   
10131                       NaN   NaN   NaN    NaN    NaN    NaN  40.0  18.5   
10361                       3.0  10.5   4.0    3.0    2.0    NaN   2.0   NaN   
11021                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   1.0   
11031                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11041                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11051                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11061                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11071                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11081                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11091                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11101                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
11111                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
14009                       2.5  14.0   1.0   13.5    5.5   18.5   7.0   NaN   
14011                       NaN   NaN   NaN    1.0    NaN    NaN   NaN   NaN   
14021                       1.0   5.5   2.0    2.0    1.0    2.0   NaN   NaN   
14031                      32.0  29.0  18.0   10.5    7.5   12.5   4.5   1.0   
14119                       6.5  13.0   7.0   24.5   13.5  113.5   NaN   NaN   
14129                       1.5   8.5   1.0   14.5    7.5   69.5   NaN   NaN   
14379                       5.5  39.0  23.0   12.0   11.5   20.0   5.0   NaN   
41101                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
41111                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
41121                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
41131                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
41161                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
51071                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
51111                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53011                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53021                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53041                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53051                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53061                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53071                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53081                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
53091                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
55011                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
55021                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
55241                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
55251                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
55261                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
56011                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
56021                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
56031                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
56191                       NaN   NaN   NaN    NaN    NaN    NaN   NaN   NaN   
56201               

In [14]:
import math

# we keep only the first digit of the stop code as it generally conforms to geographical areas.
# second digit may also be useful in some cases but could create too much statistical noise.
# for example, both CCK and BP have a mix of stop codes in both 44000 and 45000 ranges
def rounder(x):
    return int(math.floor(x/1000)) * 1000

df_round = df
df_round.ORIGIN_PT_CODE = df_round.ORIGIN_PT_CODE.apply(rounder)
df_round.DESTINATION_PT_CODE = df_round.DESTINATION_PT_CODE.apply(rounder)
df_round = df_round.groupby(["DAY_TYPE", "ORIGIN_PT_CODE", "DESTINATION_PT_CODE"]).sum()
df_round.to_csv(f"output/{service}_{direction}_grouped.csv")

In [ ]:
files = ["190_1_time.csv", "190_2_time.csv", "972_1_time.csv", "960_1_time.csv","960_2_time.csv"]
df_cons = pd.DataFrame()
for fn in files:
    df_cons = pd.concat([df_cons, pd.read_csv(f"output/{fn}")])
df_cons = df_cons.drop_duplicates()
df_cons.to_csv("btl.csv")

In [ ]:
def rounder(x):
    return int(math.floor(x/1000)) * 1000

df_round = df_cons
df_round.ORIGIN_PT_CODE = df_round.ORIGIN_PT_CODE.apply(rounder)
df_round.DESTINATION_PT_CODE = df_round.DESTINATION_PT_CODE.apply(rounder)
df_round = df_round.groupby(["DAY_TYPE", "ORIGIN_PT_CODE", "DESTINATION_PT_CODE"]).sum()
df_round.to_csv(f"btl_grouped.csv")